In [99]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

### Extract CSVs into DataFrames

In [100]:
fifa_country_file = "fifa_countries_audience.csv"
fifa_countries_df = pd.read_csv(fifa_country_file)
fifa_countries_df.head()

,country,confederation,population_share,tv_audience_share,gdp_weighted_share
0,United States,CONCACAF,4.5,4.3,11.3
1,Japan,AFC,1.9,4.9,9.1
2,China,AFC,19.5,14.8,7.3
3,Germany,UEFA,1.2,2.9,6.3
4,Brazil,CONMEBOL,2.8,7.1,5.4


In [55]:
wc_file = "WorldCups.csv"
world_cups_df = pd.read_csv(wc_file)
world_cups_df.head()

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1.045.246
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768.607


### Transform Countries DataFrame

In [82]:
# Create a filtered dataframe from specific columns
countries_cols = ["country", "population_share", "gdp_weighted_share"]
countries_new= fifa_countries_df[countries_cols].copy()
#Setting Country as index
countries_new.set_index("country", inplace=True)
countries_new.head()

,population_share,gdp_weighted_share
country,,
United States,4.5,11.3
Japan,1.9,9.1
China,19.5,7.3
Germany,1.2,6.3
Brazil,2.8,5.4


### Transform WorldCup DataFrame

In [78]:
wc_cols = ["Winner", "Runners-Up", "Third"]
worldcup_new = world_cups_df[wc_cols].copy()

In [101]:
worldcup_new=worldcup_new.rename(columns={"Runners-Up": "runner_up","Winner":"winner","Third":"third"})
#Replace incorrect country names with correct names 
worldcup_new["winner"]=worldcup_new["winner"].replace({"Germany FR":"Germany","Belgiun":"Belgium","England":"United Kingdom","USA":"United States"})
worldcup_new["runner_up"]=worldcup_new["runner_up"].replace({"Germany FR":"Germany","Belgiun":"Belgium","England":"United Kingdom","USA":"United States"})
worldcup_new["third"]=worldcup_new["third"].replace({"Germany FR":"Germany","Belgiun":"Belgium","England":"United Kingdom","USA":"United States"})
#Count the contries successes 
new_wc=worldcup_new.apply(pd.Series.value_counts)
#Replace NaN with 0
new_wc=new_wc.fillna(0)
new_wc=new_wc.rename_axis(['country'])
#Delete the country that does not exist anymore
new_wc=new_wc.drop(['Czechoslovakia'])
new_wc

,winner,runner_up,third
country,,,
Argentina,2.0,3.0,0.0
Austria,0.0,0.0,1.0
Belgium,0.0,0.0,1.0
Brazil,5.0,2.0,2.0
Chile,0.0,0.0,1.0
Croatia,0.0,1.0,1.0
France,2.0,1.0,2.0
Germany,4.0,4.0,4.0
Hungary,0.0,2.0,0.0


### Create database connection

In [102]:
connection_string = "postgres:postgres@localhost:5432/WorlCup_Fifa"
engine = create_engine(f'postgresql://{connection_string}')

In [103]:
# Confirm tables
engine.table_names()

['countries_new', 'new_wc']

### Load DataFrames into database

In [104]:
countries_new.to_sql(name='countries_new', con=engine, if_exists='append', index=True)

In [105]:
new_wc.to_sql(name='new_wc', con=engine, if_exists='append', index=True)